In [1]:
import requests
from bs4 import BeautifulSoup

In [29]:
# STAGE 1: extract the list of hyperlinks

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States"

In [3]:
resp = requests.get(url)
resp

<Response [200]>

In [4]:
resp.raise_for_status()

In [6]:
doc = BeautifulSoup(resp.text, "html.parser")

In [9]:
# print(doc.prettify())

In [13]:
tables = doc.find_all("table")
len(tables)

18

In [15]:
table = None
for t in tables:
    if len(t.find_all("tr")) >= 50:
        table = t
        break
assert(table != None)

In [20]:
trs = table.find_all("tr")
print(len(trs))

52


In [28]:
urls = []
for tr in trs[2:]:
    th = tr.find("th")
    a = th.find("a")
    urls.append("https://en.wikipedia.org" + a.attrs["href"])
urls

['https://en.wikipedia.org/wiki/Alabama',
 'https://en.wikipedia.org/wiki/Alaska',
 'https://en.wikipedia.org/wiki/Arizona',
 'https://en.wikipedia.org/wiki/Arkansas',
 'https://en.wikipedia.org/wiki/California',
 'https://en.wikipedia.org/wiki/Colorado',
 'https://en.wikipedia.org/wiki/Connecticut',
 'https://en.wikipedia.org/wiki/Delaware',
 'https://en.wikipedia.org/wiki/Florida',
 'https://en.wikipedia.org/wiki/Georgia_(U.S._state)',
 'https://en.wikipedia.org/wiki/Hawaii',
 'https://en.wikipedia.org/wiki/Idaho',
 'https://en.wikipedia.org/wiki/Illinois',
 'https://en.wikipedia.org/wiki/Indiana',
 'https://en.wikipedia.org/wiki/Iowa',
 'https://en.wikipedia.org/wiki/Kansas',
 'https://en.wikipedia.org/wiki/Kentucky',
 'https://en.wikipedia.org/wiki/Louisiana',
 'https://en.wikipedia.org/wiki/Maine',
 'https://en.wikipedia.org/wiki/Maryland',
 'https://en.wikipedia.org/wiki/Massachusetts',
 'https://en.wikipedia.org/wiki/Michigan',
 'https://en.wikipedia.org/wiki/Minnesota',
 'https

In [40]:
import os

# STAGE 2: just download the 50 pages
for url in urls:
    name = url.split("/")[-1]
    name = name.split("_(")[0]
    print(name)
    
    path = name+".html"
    if os.path.exists(path):
        continue
    
    # do the slow part, because we don't have the file
    r = requests.get(url)
    r.raise_for_status()
    
    f = open(path, "w")
    f.write(r.text)
    f.close()

Alabama
Alaska
Arizona
Arkansas
California
Colorado
Connecticut
Delaware
Florida
Georgia
Hawaii
Idaho
Illinois
Indiana
Iowa
Kansas
Kentucky
Louisiana
Maine
Maryland
Massachusetts
Michigan
Minnesota
Mississippi
Missouri
Montana
Nebraska
Nevada
New_Hampshire
New_Jersey
New_Mexico
New_York
North_Carolina
North_Dakota
Ohio
Oklahoma
Oregon
Pennsylvania
Rhode_Island
South_Carolina
South_Dakota
Tennessee
Texas
Utah
Vermont
Virginia
Washington
West_Virginia
Wisconsin
Wyoming


In [41]:
# STAGE 3: parse the files

In [50]:
def parse_state_file(path):
    f = open(path)
    html = f.read()
    f.close()
    doc = BeautifulSoup(html, "html.parser")
    stats = {}
    for tr in doc.find_all("tr"):
        cells = tr.find_all(["th", "td"])
        if len(cells) != 2:
            continue
        key = cells[0].get_text()
        val = cells[1].get_text()
        stats[key] = val
    return stats

In [52]:
stats = parse_state_file("Wisconsin.html")
stats["Capital"]

'Madison'